In [134]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
from sklearn.metrics.pairwise import cosine_similarity  # Correct import
import numpy as np


In [153]:

# Đọc dữ liệu từ tập CSV vào DataFrame
df = pd.read_csv('DATADULICHQUYNHON.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df


,Name_Hotel,original_price_list,Link,star_rating_hotel_list,Location,Convenient,Aria_locations,Comment_Hotel,User_vote_hotel,Review_Hotel,hightlight_Hotel
0,Grand Hyams Hotel - Quy Nhon Beach,3950100,https://www.booking.com/hotel/vn/grand-hyams-q...,5.0,"28 Nguyen Hue, Le Loi ward, Quy Nhon city, Bin...","Hồ bơi trong nhà, WiFi miễn phí, Phòng gia ...","Tượng Đài Chiến Thắng: 150 m, Công Viên Bàu Se...","Tram(Việt Nam): “Sang trọng, yên tĩnh, các bạ...",9.3,"Nằm ở Quy Nhơn, cách Quy Nhon Beach 2 phút đi ...","Nhìn ra biển,Nhìn ra địa danh nổi tiếng,Hồ bơi..."
1,FLC City Hotel Beach Quy Nhon,3937950,https://www.booking.com/hotel/vn/flc-beach-cit...,5.0,"An Duong Vuong Road Quy Nhon City, Binh Dinh P...","Hồ bơi trong nhà, Giáp biển, WiFi miễn phí, ...","Công Viên: 300 m, Đu Quay Uốn Lượn: 400 m, Côn...","Huy(Việt Nam): “Phòng rộng, view đẹp, vị trí ...",8.6,"Nằm giáp biển, FLC City Hotel Beach Quy Nhon c...","Nhìn ra biển, Địa điểm hàng đầu: Được khách gầ..."
2,RAON Hotel - STAY 24H,1426513,https://www.booking.com/hotel/vn/nam-thu.vi.ht...,4.0,"09 Nguyễn Thị Định, Quy Nhơn, Việt Nam – Vị t...","Hồ bơi ngoài trời, Giáp biển, WiFi miễn phí,...","Công Viên: 500 m, Công Viên: 600 m, Đu Quay Uố...","Gabriel(Việt Nam): “Mọi thứ đều ổn, sạch sẽ, ...",9.3,"Tọa lạc ở Quy Nhơn, cách Quy Nhon Beach 7 phút...","Hồ bơi có tầm nhìn,Hướng nhìn ra đường phố yên..."
3,MENTO HOTEL QUY NHƠN,1863000,https://www.booking.com/hotel/vn/mento-quy-nho...,3.0,"Nguyễn Huệ 409C - 409D, Quy Nhơn, Việt Nam – ...","3 hồ bơi, Giáp biển, WiFi miễn phí, Xe đưa đ...","Ngã 6 Ngô Mây: 250 m, Công Viên: 400 m, Đu Qua...",No comment,8.4,"Nằm giáp biển, MENTO HOTEL QUY NHƠN cung cấp c...","Nhìn ra thành phố,Nhìn ra núi,Nhìn ra địa danh..."
4,Moon's House Homestay Quy Nhơn,1050000,https://www.booking.com/hotel/vn/moon-39-s-hou...,2.0,"82 Vũ Bảo, Quy Nhơn, Việt Nam – Vị trí xuất s...","WiFi miễn phí, Xe đưa đón sân bay, Phòng g...","Ngã 6 Ngô Mây: 350 m, Công Viên: 550 m, Đu Qua...","Vy(Việt Nam): “Không gian vừa đủ, sạch sẽ, mề...",9.5,Nằm ở Quy Nhơn và cách Quy Nhon Beach chưa đến...,"Sân hiên, Địa điểm hàng đầu: Được khách gần đâ..."
...,...,...,...,...,...,...,...,...,...,...,...
438,Q House- Green Villa Quy Nhơn,8967240,https://www.booking.com/hotel/vn/q-house-homes...,0.0,"70/14/12 70/14/13 Han Mac Tu, Ghenh Rang Ward,...","WiFi miễn phí, Xe đưa đón sân bay, Phòng g...","Công Viên: 600 m, Công Viên: 1,1 km, Đu Quay U...",No comment,9.0,"Cách Quy Nhon Beach 4 phút đi bộ, Q House- Gre...","Nhìn ra thành phố,Sân hiên,Nhìn ra vườn,Hướng ..."
439,Căn hộ du lịch Quy Nhơn giá rẻ-view biển 7,2904000,https://www.booking.com/hotel/vn/quy-nhon-colo...,0.0,"Chung cư An Phú Thịnh Garden Tower, Khu đô thị...","WiFi miễn phí, Chỗ đỗ xe miễn phí","Công Viên Đống Đa: 2,7 km, Công Viên Đống Đa: ...",No comment,8.4,"Tọa lạc ở Quy Nhơn thuộc Bình Định, Căn hộ du ...",Sân hiênNo hightlight
440,Quy Nhon Homestay BnB,8997000,https://www.booking.com/hotel/vn/quynhonhomest...,0.0,"20 Trần Kỳ Phong, Quy Nhơn, Bình Định, Quy Nhơ...","Giáp biển, WiFi miễn phí, Xe đưa đón sân ba...","Công Viên: 500 m, Công Viên: 900 m, Đu Quay Uố...",Hồng(Việt Nam): “Hàng xóm thân thiện! Địa điể...,8.4,Nằm ở Quy Nhơn và cách Quy Nhon Beach 6 phút đ...,"Nhìn ra thành phố,Nhìn ra núi,Sân hiên,Hướng n..."
441,CĂN HỘ QUY NHƠN,5400000,https://www.booking.com/hotel/vn/can-ho-quy-nh...,0.0,"76 Trần Hưng Đạo 26, Quy Nhơn, Việt Nam – Hiể...","Hồ bơi, WiFi miễn phí, Chỗ đỗ xe miễn phí","Tượng Đài Chiến Thắng: 750 m, Công Viên Đống Đ...",No comment,8.4,No review,Sân hiênNo hightlight


In [59]:
df.columns

Index(['Name_Hotel', 'original_price_list', 'Link', 'star_rating_hotel_list',
       'Location', 'Convenient', 'Aria_locations', 'Comment_Hotel',
       'User_vote_hotel', 'Review_Hotel', 'hightlight_Hotel'],
      dtype='object')

In [91]:
df.isnull().sum()

Name_Hotel                0
original_price_list       0
Link                      0
star_rating_hotel_list    0
Location                  0
Convenient                0
Aria_locations            0
Comment_Hotel             0
User_vote_hotel           0
Review_Hotel              0
hightlight_Hotel          0
description               0
numeric_values            0
dtype: int64

In [85]:
from sklearn.preprocessing import StandardScaler
numeric_columns = ['star_rating_hotel_list', 'User_vote_hotel']
df[numeric_columns] = StandardScaler().fit_transform(df[numeric_columns])

In [154]:
from sklearn.preprocessing import MinMaxScaler

# Chuyển đổi cột 'star_rating_hotel_list' sang kiểu số và sau đó chuẩn hóa
df['star_rating_hotel_list'] = df['star_rating_hotel_list'].astype(float)
scaler_star_rating = MinMaxScaler()
df['star_rating_hotel_list_normalized'] = scaler_star_rating.fit_transform(df[['star_rating_hotel_list']])

# Chuyển đổi cột 'original_price_list' sang kiểu số và sau đó chuẩn hóa
df['original_price_list'] = df['original_price_list'].astype(float)
scaler_original_price = MinMaxScaler()
df['original_price_list_normalized'] = scaler_original_price.fit_transform(df[['original_price_list']])

# Chuyển đổi cột 'User_vote_hotel' sang kiểu số và sau đó chuẩn hóa
df['User_vote_hotel'] = df['User_vote_hotel'].astype(float)
scaler_user_vote = MinMaxScaler()
df['User_vote_hotel_normalized'] = scaler_user_vote.fit_transform(df[['User_vote_hotel']])

df['description'] = (
    df['star_rating_hotel_list_normalized'].astype(str) +
    ' ' +
    df['original_price_list_normalized'].astype(str) +
    ' ' +
    df['User_vote_hotel_normalized'].astype(str)
)

# Sử dụng TF-IDF để chuyển đổi văn bản thành vector
query_hotel = 'Grand Hyams Hotel - Quy Nhon Beach'

# Kiểm tra xem tên khách sạn có tồn tại trong DataFrame hay không
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'])

# Sử dụng PCA để giảm chiều dữ liệu
num_components = 2
pca = PCA(n_components=num_components)
pca_result = pca.fit_transform(tfidf_matrix.toarray())

# Kiểm tra xem tên khách sạn có tồn tại trong DataFrame hay không
if query_hotel in df['Name_Hotel'].values:
    # Tìm index của khách sạn mẫu
    query_index = df[df['Name_Hotel'] == query_hotel].index[0]

    # Hiển thị giá trị của thành phần chính đầu tiên và thứ hai của khách sạn mẫu
    print(f"Principal Component 1: {pca_result[query_index, 0]}")
    print(f"Principal Component 2: {pca_result[query_index, 1]}")

    # Tính độ tương tự cosine giữa khách sạn mẫu và tất cả các khách sạn
    similarity_scores = cosine_similarity(pca_result[query_index].reshape(1, -1), pca_result)

    # Hiển thị giá trị độ tương tự cosine
    print("Similarity Scores:")
    print(similarity_scores)

    # Lấy 10 khách sạn có độ tương tự cao nhất
    similar_hotels_indices = similarity_scores.argsort()[0][::-1][1:11]
    similar_hotels = df.iloc[similar_hotels_indices]

    # In tên khách sạn của top 10
    top_10_hotels_names = similar_hotels['Name_Hotel'].tolist()
    print("Top 10 Khách sạn có độ tương tự cao nhất:")
    print(top_10_hotels_names)
else:
    print(f"Không tìm thấy khách sạn '{query_hotel}' trong DataFrame.")

Principal Component 1: -0.12316378753576318
Principal Component 2: -0.06509630112565626
Similarity Scores:
[[ 1.00000000e+00  9.58534078e-01  1.00000000e+00 -7.23235855e-01
   9.39162889e-01 -6.66176054e-01 -6.56727684e-01  9.93983322e-01
   9.95856731e-01  6.91322842e-01 -8.92332129e-01  9.91589227e-01
   9.75996948e-01  9.17974459e-01  9.60758585e-01 -6.56727684e-01
  -6.54504788e-01  9.60758585e-01  9.53054090e-01  6.91322842e-01
   9.53054090e-01  6.69510166e-01 -7.01090359e-01 -7.43001282e-01
   9.80435743e-01  9.93967172e-01  9.62815681e-01  9.17974459e-01
   3.02420875e-02  8.10418627e-02  9.66965943e-01  9.94070590e-01
  -6.56727684e-01  9.95856731e-01  2.77653834e-02  9.27409242e-01
   6.10426550e-02 -6.56727684e-01  8.94324450e-02 -6.19180547e-01
  -6.43929486e-01  9.69764232e-01  9.60758585e-01  9.94729015e-01
   8.41108302e-01  9.67319804e-01  9.99825128e-01  6.10426550e-02
   9.65633343e-01 -6.56727684e-01  9.95856731e-01  9.98294254e-01
   9.93983322e-01  9.66904050e-01 -

In [125]:
print(cosine_similarities_combined)#PCA

[[ 1.  1.  1. ... -1. -1.  1.]
 [ 1.  1.  1. ... -1. -1.  1.]
 [ 1.  1.  1. ... -1. -1.  1.]
 ...
 [-1. -1. -1. ...  1.  1. -1.]
 [-1. -1. -1. ...  1.  1. -1.]
 [ 1.  1.  1. ... -1. -1.  1.]]


In [143]:
# Xử lý dữ liệu, nếu cần
# Ví dụ: Kết hợp các trường 'Tên khách sạn' và 'Tiện ích' thành một trường duy nhất 'Mô tả'
#df['Mô tả'] = df['Name_Hotel'] + ' ' + df['star_rating_hotel_list'] + ' ' + df['voting_user_list'] + ' ' + df['review_list']
numeric_columns = ['star_rating_hotel_list', 'User_vote_hotel','original_price_list']

# Tạo cột 'description' chỉ từ các cột giá trị số
df['description'] = df[numeric_columns].astype(str).agg(' '.join, axis=1)

# Sử dụng TF-IDF để biểu diễn văn bản thành ma trận số
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'].fillna(''))

# Tính cosine similarity giữa các khách sạn
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Hàm đề xuất dựa trên cosine similarity
def content_based_recommendation(hotel_name, cosine_similarities, df):
    hotel_index = df[df['Name_Hotel'] == hotel_name].index
    if not hotel_index.empty:
        hotel_index = hotel_index[0]
        similar_hotels = list(enumerate(cosine_similarities[hotel_index]))
        similar_hotels = sorted(similar_hotels, key=lambda x: x[1], reverse=True)
        similar_hotels = similar_hotels[1:11]  # Lấy 10 khách sạn tương tự

        # Lấy tất cả giá trị từ danh sách similar_hotels
        recommended_hotels = [df.iloc[hotel[0]]['Name_Hotel'] for hotel in similar_hotels]
        return recommended_hotels
    else:
        print(f'Hotel not found: {hotel_name}')
        return []

# Đề xuất 10 khách sạn tương tự với khách sạn có tên là 'Tên khách sạn cần đề xuất'
recommended_hotels = content_based_recommendation("Grand Hyams Hotel - Quy Nhon Beach", cosine_similarities, df)
print(recommended_hotels)

['Ánh Dương Homestay - Coastal stop - Just 2 mins walking to Eo Gio', 'NX Home', 'FLC Studio Beach Quy Nhơn', 'TMS Pullman Quy Nhơn Căn Hộ - Xuân Phương Apartment', 'FLC Sea Tower Quy Nhơn - Lâm Phong (Căn hộ 2 Phòng ngủ)', 'RAON Hotel - STAY 24H', 'FLC Sea Tower Quy Nhơn Bình Định-Seaside city Center', 'TMS Quy Nhơn - Lee Xinh', 'Tony Apartment -FLC Sea Tower Quy Nhon', 'TMS Apartment View Biển Luxury Quy Nhơn']


In [144]:
print(cosine_similarities)

[[1.         0.43105516 0.6501968  ... 0.         0.         0.59203595]
 [0.43105516 1.         0.         ... 0.         0.         0.59836644]
 [0.6501968  0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         0.32062355]
 [0.         0.         0.         ... 1.         1.         0.32062355]
 [0.59203595 0.59836644 0.         ... 0.32062355 0.32062355 1.        ]]


In [148]:
import pandas as pd
hotels_to_check = top_10_hotels_names


# Create an empty DataFrame
result_df = pd.DataFrame(columns=['Hotel', 'Target Hotel', 'Similarity Score', 'Information'])

# Target hotel for comparison
target_hotel = "Grand Hyams Hotel - Quy Nhon Beach"

# Check the similarities between the specified hotels and the target hotel
for hotel_name in hotels_to_check:
    # Check if both hotels are in the DataFrame
    if hotel_name in df['Name_Hotel'].values and target_hotel in df['Name_Hotel'].values:
        # Calculate the cosine similarity between the two hotels
        hotel_index = df[df['Name_Hotel'] == hotel_name].index[0]
        target_index = df[df['Name_Hotel'] == target_hotel].index[0]
        similarity_score = cosine_similarities[hotel_index, target_index]

        # Get the information for each hotel
        hotel_info = df.iloc[hotel_index].to_dict()
        target_info = df.iloc[target_index].to_dict()

        # Append information to the result DataFrame
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': similarity_score,
            'Information': f"{hotel_name}: {hotel_info}\n{target_hotel}: {target_info}"
        }, ignore_index=True)
    else:
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': 'Not found',
            'Information': 'At least one of the hotels not found'
        }, ignore_index=True)

# Display the result DataFrame
result_df


,Hotel,Target Hotel,Similarity Score,Information
0,Ánh Dương Homestay - Coastal stop - Just 2 min...,Grand Hyams Hotel - Quy Nhon Beach,1.000000,Ánh Dương Homestay - Coastal stop - Just 2 min...
1,Anantara Quy Nhon Villas,Grand Hyams Hotel - Quy Nhon Beach,0.461787,Anantara Quy Nhon Villas: {'Name_Hotel': 'Anan...
2,FLC Grand Hotel Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.461787,FLC Grand Hotel Quy Nhon: {'Name_Hotel': 'FLC ...
3,Hoang Yen Canary Hotel,Grand Hyams Hotel - Quy Nhon Beach,0.000000,Hoang Yen Canary Hotel: {'Name_Hotel': 'Hoang ...
4,Tân Thi Hotel,Grand Hyams Hotel - Quy Nhon Beach,0.000000,"Tân Thi Hotel: {'Name_Hotel': 'Tân Thi Hotel',..."
5,ND Travel - FLC Sea Tower Quy Nhơn,Grand Hyams Hotel - Quy Nhon Beach,0.000000,ND Travel - FLC Sea Tower Quy Nhơn: {'Name_Hot...
6,Delight Hotel,Grand Hyams Hotel - Quy Nhon Beach,0.000000,"Delight Hotel: {'Name_Hotel': 'Delight Hotel',..."
7,SUSU HOTEL Quy Nhơn Center Near Sea,Grand Hyams Hotel - Quy Nhon Beach,0.000000,SUSU HOTEL Quy Nhơn Center Near Sea: {'Name_Ho...
8,Mira Bãi Xếp Quy Nhơn - The Hidden Jewel,Grand Hyams Hotel - Quy Nhon Beach,0.000000,Mira Bãi Xếp Quy Nhơn - The Hidden Jewel: {'Na...
9,HaTa Hotel,Grand Hyams Hotel - Quy Nhon Beach,0.000000,"HaTa Hotel: {'Name_Hotel': 'HaTa Hotel', 'orig..."


In [147]:
import pandas as pd
hotels_to_check = recommended_hotels

# Create an empty DataFrame
result_df = pd.DataFrame(columns=['Hotel', 'Target Hotel', 'Similarity Score', 'Information'])

# Target hotel for comparison
target_hotel = "Grand Hyams Hotel - Quy Nhon Beach"

# Check the similarities between the specified hotels and the target hotel
for hotel_name in hotels_to_check:
    # Check if both hotels are in the DataFrame
    if hotel_name in df['Name_Hotel'].values and target_hotel in df['Name_Hotel'].values:
        # Calculate the cosine similarity between the two hotels
        hotel_index = df[df['Name_Hotel'] == hotel_name].index[0]
        target_index = df[df['Name_Hotel'] == target_hotel].index[0]
        similarity_score = cosine_similarities[hotel_index, target_index]

        # Get the information for each hotel
        hotel_info = df.iloc[hotel_index].to_dict()
        target_info = df.iloc[target_index].to_dict()

        # Append information to the result DataFrame
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': similarity_score,
            'Information': f"{hotel_name}: {hotel_info}\n{target_hotel}: {target_info}"
        }, ignore_index=True)
    else:
        result_df = result_df.append({
            'Hotel': hotel_name,
            'Target Hotel': target_hotel,
            'Similarity Score': 'Not found',
            'Information': 'At least one of the hotels not found'
        }, ignore_index=True)

# Display the result DataFrame
result_df


,Hotel,Target Hotel,Similarity Score,Information
0,Ánh Dương Homestay - Coastal stop - Just 2 min...,Grand Hyams Hotel - Quy Nhon Beach,1.000000,Ánh Dương Homestay - Coastal stop - Just 2 min...
1,NX Home,Grand Hyams Hotel - Quy Nhon Beach,0.716157,"NX Home: {'Name_Hotel': 'NX Home', 'original_p..."
2,FLC Studio Beach Quy Nhơn,Grand Hyams Hotel - Quy Nhon Beach,0.716157,FLC Studio Beach Quy Nhơn: {'Name_Hotel': 'FLC...
3,TMS Pullman Quy Nhơn Căn Hộ - Xuân Phương Apar...,Grand Hyams Hotel - Quy Nhon Beach,0.716157,TMS Pullman Quy Nhơn Căn Hộ - Xuân Phương Apar...
4,FLC Sea Tower Quy Nhơn - Lâm Phong (Căn hộ 2 P...,Grand Hyams Hotel - Quy Nhon Beach,0.716157,FLC Sea Tower Quy Nhơn - Lâm Phong (Căn hộ 2 P...
5,RAON Hotel - STAY 24H,Grand Hyams Hotel - Quy Nhon Beach,0.650197,RAON Hotel - STAY 24H: {'Name_Hotel': 'RAON Ho...
6,FLC Sea Tower Quy Nhơn Bình Định-Seaside city ...,Grand Hyams Hotel - Quy Nhon Beach,0.650197,FLC Sea Tower Quy Nhơn Bình Định-Seaside city ...
7,TMS Quy Nhơn - Lee Xinh,Grand Hyams Hotel - Quy Nhon Beach,0.641177,TMS Quy Nhơn - Lee Xinh: {'Name_Hotel': 'TMS Q...
8,Tony Apartment -FLC Sea Tower Quy Nhon,Grand Hyams Hotel - Quy Nhon Beach,0.592036,Tony Apartment -FLC Sea Tower Quy Nhon: {'Name...
9,TMS Apartment View Biển Luxury Quy Nhơn,Grand Hyams Hotel - Quy Nhon Beach,0.592036,TMS Apartment View Biển Luxury Quy Nhơn: {'Nam...
